In [1]:
import os
import time
import certifi
import requests
import pandas as pd
import xml.etree.ElementTree as ET

# Usa el bundle de certifi para evitar errores de certificado en requests
os.environ.setdefault("REQUESTS_CA_BUNDLE", certifi.where())
os.environ.setdefault("SSL_CERT_FILE", certifi.where())


'/home/pablo/.local/lib/python3.10/site-packages/certifi/cacert.pem'

In [2]:
VERIFY_SSL = os.getenv("OAI_VERIFY_SSL", "false").lower() == "true"
CA_BUNDLE = os.getenv("OAI_CA_BUNDLE") or certifi.where()

# Pon VERIFY_SSL=True si quieres validar el certificado con CA_BUNDLE; se desactiva por defecto


In [3]:
base_url = catalog.load('params:oai_fetch_options.base_url')
context = catalog.load('params:oai_fetch_options.context')
env = 'prod'
metadata_format = 'oai_dc'


                    INFO     Loading data from params:oai_fetch_options.base_url                ]8;id=129806;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=252775;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

                    INFO     Loading data from params:oai_fetch_options.context                 ]8;id=412328;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=953450;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (MemoryDataset)...                                                                    

In [4]:
df = catalog.load('dv_oai/dim_oai_set')
df

                    INFO     Loading data from dv_oai/dim_oai_set (SQLTableDataset)...          ]8;id=745538;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=944521;file:///home/pablo/.local/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,set_id,name,set_hk
0,col_11336_279,Articulos(CCT - SAN LUIS),b'\x00\x01\xc4\xd7\xe6\xf9\xe7]\x11\xdb\x13\x9...
1,col_11336_134184,Datos de Investigación(CEUR),b'\x00\x15\xd27IQ\xae!4evc\xa7\xce\x11\x80'
2,col_11336_43,Articulos(IGEHCS),b'\x00bM\xfdse\x06\xc9\x17#*\xbd\x9a\r\xd9\r'
3,col_11336_90048,Libros(OCA HOUSSAY),b'\x00{\x9b\xdf#n&U\xc9\xc5\x85\x1cU\xcc\xb1{'
4,col_11336_90026,Libros(IQUIBA-NEA),"b'\x00\x90\x0c/nM\x00\xb5\xa34-\xa3H\xea,\x00'"
...,...,...,...
2439,col_11336_134379,Datos de Investigación (INDES),"b'\xff!\xfdZ\xe865R`k\x98\xfa""f\x1c\xb2'"
2440,col_11336_432,Articulos(IIBBA),b'\xff*i\x99\x86\x070\x8aGf\x8a\xa5L\x0b\x13\xdd'
2441,col_11336_90059,Libros(UMYMFOR),b'\xff\x90\xd5\x02\xed\xb1G)\x80\xd3\x98E\xe2\...
2442,col_11336_90303,Capítulos de libros(INIGEM),b'\xff\xcf\xcd~\xa2h\x81\xd0\x17\x93f&\x17\xc9...


In [5]:
set_id = df.iloc[0,0]
set_id

'col_11336_279'

In [6]:
def get_oai_records(base_url, verify=None):
    start_time = time.time()

    verify_param = CA_BUNDLE if VERIFY_SSL else False
    if verify is not None:
        verify_param = verify

    response = requests.get(base_url, verify=verify_param)
    end_time = time.time()
    elapsed_time = end_time - start_time

    print(f"Sleeping for {elapsed_time:.2f} seconds")
    time.sleep(elapsed_time)

    if response.status_code == 200:
        return response
    else:
        print(f"Error: {response.status_code}")
        return None


In [7]:
def oai_extract_item_by_set(base_url: str, context: str, set_id: str, metadata_format: str, env: str, verify=None) -> pd.DataFrame:
    records = []
    resumption_token = 0
    iteration_limit = 2
    iteration_count = 0
   
    while True:
        if env == 'dev' and iteration_count >= iteration_limit:
            break

        params = f'/{context}?verb=ListRecords&resumptionToken={metadata_format}///{set_id}/{resumption_token}'
        url = base_url + params
        
        print(f"Consultando: {url}")
        
        response = get_oai_records(url, verify=verify)

        resumption_token += 100
        iteration_count += 1

        if not response or not response.ok:
            print(f"Error al consultar: {url}")
            break

        xml_content = response.text
        root = ET.fromstring(xml_content)
        ns = {
            'oai': 'http://www.openarchives.org/OAI/2.0/',
            'dc': 'http://purl.org/dc/elements/1.1/'
        }

        record_nodes = root.findall('.//oai:record', ns)


        if not record_nodes:
            print("No se encontraron más registros.")
            break

        for record in record_nodes:
            identifier = record.find('.//oai:identifier', ns)
            item_id = identifier.text if identifier is not None else None
            metadata = record.find('.//oai:metadata', ns)

            if metadata is None:
                continue

            # Valores simples
            title = metadata.find('.//dc:title', ns)
            date = metadata.find('.//dc:date', ns)

            # Multivaluados
            creators = [e.text for e in metadata.findall('.//dc:creator', ns)]
            types = [e.text for e in metadata.findall('.//dc:type', ns)]
            identifiers = [e.text for e in metadata.findall('.//dc:identifier', ns)]
            languages = [e.text for e in metadata.findall('.//dc:language', ns)]
            publishers = [e.text for e in metadata.findall('.//dc:publisher', ns)]
            subjects = [e.text for e in metadata.findall('.//dc:subject', ns)]
            relations = [e.text for e in metadata.findall('.//dc:relation', ns)]
            rights = [e.text for e in metadata.findall('.//dc:rights', ns)]

            records.append({
                'item_id': item_id,
                'col_id': set_id,
                'title': title.text if title is not None else None,
                'date': date.text if date is not None else None,
                'creators': creators,
                'types': types,
                'identifiers': identifiers,
                'languages': languages,
                'subjects': subjects,
                'publishers': publishers,
                'relations': relations,
                'rights': rights
            })

    df = pd.DataFrame(records)

    return df, df.head(100)


In [13]:
df, df_dev = oai_extract_item_by_set(base_url, context, set_id, metadata_format, env)

Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_279/0


[11/22/25 21:01:22] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=704204;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=762888;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.31 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_279/100


                    WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=251661;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=90711;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.31 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_279/200


...

Sleeping for 0.82 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_279/800


[11/22/25 21:01:36] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=887038;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=888399;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.75 seconds
Consultando: https://ri.conicet.gov.ar/oai//request?verb=ListRecords&resumptionToken=oai_dc///col_11336_279/900


[11/22/25 21:01:37] WARNING  /home/pablo/.local/lib/python3.10/site-packages/urllib3/connectionpool ]8;id=752838;file:///usr/lib/python3.10/warnings.py\warnings.py]8;;\:]8;id=573412;file:///usr/lib/python3.10/warnings.py#109\109]8;;\
                             .py:1099: InsecureRequestWarning: Unverified HTTPS request is being                   
                             made to host 'ri.conicet.gov.ar'. Adding certificate verification is                  
                             strongly advised. See:                                                                
                             https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warni                
                             ngs                                                                                   
                               warnings.warn(                                                                      
                                                                                                                   

Sleeping for 0.18 seconds
No se encontraron más registros.


In [14]:
df

,item_id,col_id,title,date,creators,types,identifiers,languages,subjects,publishers,relations,rights
0,oai:ri.conicet.gov.ar:11336/215406,col_11336_279,Estrategias de fertilización nitrogenada para ...,2019-07,"[Dillchneider Loza, Alexandra, Frasier, Ileana...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/215406, Dillchnei...",[spa],"[TEXTURA DEL SUELO, EFICIENCIA DE USO DEL NITR...",[Universidad Nacional de La Pampa. Facultad de...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
1,oai:ri.conicet.gov.ar:11336/140529,col_11336_279,3D litho-constrained inversion model of southe...,2019-04-05,"[Christiansen, Rodolfo Omar, Morosini, Augusto...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/140529, Christian...",[eng],"[DOUBLY-VERGENT STRUCTURE, FAMATINIAN COLLISIO...",[Elsevier Science],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
2,oai:ri.conicet.gov.ar:11336/140442,col_11336_279,Efecto de la disponibilidad de agua sobre una ...,2019-04,"[Rauber, Ruth Bibiana, Demaría, Manuel, Steina...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/140442, Rauber, R...",[spa],"[Invasión, Fenología, Precipitaciones, https:/...",[Asociación Argentina de Ecología],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
3,oai:ri.conicet.gov.ar:11336/261114,col_11336_279,"3D model of the El Hornito pluton, Sierras Pam...",2024-08,"[Muñoz, Brian Lucas, Enriquez, Eliel, Christia...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/261114, Muñoz, Br...",[eng],"[GRANITE, EMPLACEMENT, 3D MODEL, GRAVITY DATA,...",[Pergamon-Elsevier Science Ltd],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http..."
4,oai:ri.conicet.gov.ar:11336/94498,col_11336_279,La convergencia asistencia/seguridad: trama de...,2018-12,"[Seveso Zanin, Emilio José, Peano, Alejandra d...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/94498, Seveso Zan...",[spa],"[POLÍTICA SOCIAL, Seguridad, Tterritorio, Pobr...",[Universidad Nacional de Cuyo. Facultad de Fil...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
...,...,...,...,...,...,...,...,...,...,...,...,...
840,oai:ri.conicet.gov.ar:11336/151963,col_11336_279,Manifestaciones rupestres e interacciones regi...,2021-05,"[Curtoni, Rafael Pedro, Oliván, Augusto Santia...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/151963, Curtoni, ...",[spa],"[TILISARAO, PAISAJE, ARTE RUPESTRE, MOVILIDAD ...",[Instituto Nacional de Antropología y Pensamie...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
841,oai:ri.conicet.gov.ar:11336/219610,col_11336_279,Filling gaps in the phylogeny of Amauroderma s...,2023-04,"[Peres. R.S., Roblego, G, L., Robledo , G., Dr...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/219610, Peres. R....",[eng],"[AFRICA, FURTADOMYCES, GANODERMATACEAE, NEOTRO...",[Elsevier Science],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http..."
842,oai:ri.conicet.gov.ar:11336/26777,col_11336_279,Dynamic function of the Alkyl spacer of acetog...,2013-11,"[Bombasaro, José Abel, Barrera Guisasola, Exeq...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/26777, Bombasaro,...",[eng],"[Acetogenins, Molecular Dynamics, Sar, Decane/...",[Bentham Science Publishers],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
843,oai:ri.conicet.gov.ar:11336/148622,col_11336_279,El escenario para la construcción de políticas...,2019-07,"[Juliá, Marta Susana, Perez Cubero, María 

In [15]:
df_dev

,item_id,col_id,title,date,creators,types,identifiers,languages,subjects,publishers,relations,rights
0,oai:ri.conicet.gov.ar:11336/215406,col_11336_279,Estrategias de fertilización nitrogenada para ...,2019-07,"[Dillchneider Loza, Alexandra, Frasier, Ileana...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/215406, Dillchnei...",[spa],"[TEXTURA DEL SUELO, EFICIENCIA DE USO DEL NITR...",[Universidad Nacional de La Pampa. Facultad de...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
1,oai:ri.conicet.gov.ar:11336/140529,col_11336_279,3D litho-constrained inversion model of southe...,2019-04-05,"[Christiansen, Rodolfo Omar, Morosini, Augusto...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/140529, Christian...",[eng],"[DOUBLY-VERGENT STRUCTURE, FAMATINIAN COLLISIO...",[Elsevier Science],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
2,oai:ri.conicet.gov.ar:11336/140442,col_11336_279,Efecto de la disponibilidad de agua sobre una ...,2019-04,"[Rauber, Ruth Bibiana, Demaría, Manuel, Steina...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/140442, Rauber, R...",[spa],"[Invasión, Fenología, Precipitaciones, https:/...",[Asociación Argentina de Ecología],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
3,oai:ri.conicet.gov.ar:11336/261114,col_11336_279,"3D model of the El Hornito pluton, Sierras Pam...",2024-08,"[Muñoz, Brian Lucas, Enriquez, Eliel, Christia...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/261114, Muñoz, Br...",[eng],"[GRANITE, EMPLACEMENT, 3D MODEL, GRAVITY DATA,...",[Pergamon-Elsevier Science Ltd],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/restrictedAccess, http..."
4,oai:ri.conicet.gov.ar:11336/94498,col_11336_279,La convergencia asistencia/seguridad: trama de...,2018-12,"[Seveso Zanin, Emilio José, Peano, Alejandra d...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/94498, Seveso Zan...",[spa],"[POLÍTICA SOCIAL, Seguridad, Tterritorio, Pobr...",[Universidad Nacional de Cuyo. Facultad de Fil...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
...,...,...,...,...,...,...,...,...,...,...,...,...
95,oai:ri.conicet.gov.ar:11336/60450,col_11336_279,The aftermath of the Fukushima nuclear acciden...,2016-03,"[Gallardo, Adrian Hugo, Marui, Atsunao]","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/60450, Gallardo, ...",[eng],"[Control Measures, Fukushima, Groundwater Cont...",[Elsevier Science],[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
96,oai:ri.conicet.gov.ar:11336/158662,col_11336_279,La Psicología Positiva en la revista Psicodeba...,2010-12,"[Mariñelarena-Dondena, Luciana, Klappenbach, H...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/158662, Mariñelar...",[spa],"[PSICOLOGÍA POSITIVA, HISTORIA DE LA PSICOLOGÍ...",[Universidad de Palermo. Facultad de Ciencias ...,[info:eu-repo/semantics/altIdentifier/url/http...,"[info:eu-repo/semantics/openAccess, https://cr..."
97,oai:ri.conicet.gov.ar:11336/7137,col_11336_279,Androgen receptor expression in pituitary of m...,2015-03,"[Filippa, Veronica Palmira, Rosales, Gabriela ...","[info:eu-repo/semantics/article, info:ar-repo/...","[http://hdl.handle.net/11336/7137, Filippa, Ve...",[eng],"[LAGOSTOMUS, PITUITARY, ANDROGEN, RECEPTOR, ht...",[Hindawi Publishing Corporation],"[info:eu-repo/semantics/altIdentifier/doi/, in...","[info:eu-repo/semantics/openAccess, https://cr..."
98,oai:ri.conicet.gov.ar:11336/156404,col_11336_279,El psicoanálisis en los debates sobre el rol d...,2000-12,"[Klappenbach, Hugo Alberto Arturo]","[info:eu-repo/s